In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import bs4
import time
import random


import json
import os
from dotenv import load_dotenv
load_dotenv()


ณิชนันท์ ประจักษ์


In [3]:
URL = os.getenv("URL")
delay = 2

def open_website():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-tensorflow')  # ปิดการใช้งาน TensorFlow
    options.add_argument('--disable-software-rasterizer')  # ปิดการประมวลผลกราฟฟิก
    options.add_experimental_option("detach", True)
    options.add_experimental_option("excludeSwitches", ['enable-automation'])
    driver = webdriver.Chrome(options=options)

    driver.get(URL)

    time.sleep(delay)
    driver.execute_script("document.body.style.zoom='70%'")
    return driver

driver = open_website()


In [4]:
"""

กรอก Input Text

"""

# Load environment variables from .env file
SCHOOL = os.getenv('SCHOOL')
NUMBER_STUDENT = os.getenv('NUMBER_STUDENT')
PREFIX = os.getenv('PREFIX')
NAME = os.getenv('NAME')
CLASS = os.getenv('CLASS')
NO = os.getenv('NO')

# ค่าที่ต้องกรอกลงในแต่ละช่อง
input_data = {
    "ชื่อโรงเรียน": SCHOOL,
    "เลขประจำตัวนักเรียน": NUMBER_STUDENT,
    "คำนำหน้าชื่อ": PREFIX,
    "ชื่อ-สกุล": NAME,
    "ระดับชั้น": CLASS,
    "เลขที่": NO
}

# ดึงข้อมูลหน้าเว็บ
data = driver.page_source
soup = bs4.BeautifulSoup(data, "html.parser")
list_items = soup.select(
    '#mG61Hd > div.RH5hzf.RLS9Fe > div.lrKTG > div.o3Dpx > div')

for index, item in enumerate(list_items, 1):
    text = item.text.split(" ")[0]  # ดึงเฉพาะข้อความหัวข้อ
    # Check text
    text_inputs = driver.find_elements(By.CSS_SELECTOR,
                                f"#mG61Hd > div.RH5hzf.RLS9Fe > div.lrKTG > div.o3Dpx > div:nth-child({index}) input[type='text']")
    if text_inputs:
        text_inputs[0].send_keys(input_data[text])
        print(f'✅ กรอกข้อมูลช่อง "{text}" เสร็จ')


✅ กรอกข้อมูลช่อง "ชื่อโรงเรียน" เสร็จ
✅ กรอกข้อมูลช่อง "เลขประจำตัวนักเรียน" เสร็จ
✅ กรอกข้อมูลช่อง "ชื่อ-สกุล" เสร็จ
✅ กรอกข้อมูลช่อง "เลขที่" เสร็จ


In [5]:
"""

กรอก Input Radio, Dropdown

"""

# ดึงข้อมูลหน้าเว็บ
data = driver.page_source
soup = bs4.BeautifulSoup(data, "html.parser")
list_items = soup.select(
    '#mG61Hd > div.RH5hzf.RLS9Fe > div.lrKTG > div.o3Dpx > div')

for index, item in enumerate(list_items, 1):
    text = item.text.split(" ")[0]  # ดึงเฉพาะข้อความหัวข้อ

    # ตรวจสอบ Dropdown ของ index ปัจจุบัน
    dropdowns = driver.find_elements(By.CSS_SELECTOR, 
        f"#mG61Hd > div.RH5hzf.RLS9Fe > div.lrKTG > div.o3Dpx > div:nth-child({index}) .MocG8c.HZ3kWc.mhLiyf.LMgvRb.KKjvXb.DEh1R")

    if dropdowns:
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", dropdowns[0])
        time.sleep(0.5)  # ให้เวลา UI โหลด

        print(f"📌 พบ Dropdown ในช่อง: {text}")

        # คลิกเปิด dropdown
        driver.execute_script("arguments[0].click();", dropdowns[0])
        time.sleep(1)

        # เลือกตัวเลือกอื่นใน radio dropdown other
        options = driver.find_elements(By.CSS_SELECTOR, 
            f"#mG61Hd > div.RH5hzf.RLS9Fe > div.lrKTG > div.o3Dpx > div:nth-child({index}) div[role='option']")
        
        valid_options = [
            opt for opt in options if opt.text.strip() != "เลือก"]
        
        # เลือกค่าจาก .env หากมี
        if text in input_data or options:
            selected_option = None
            for op in valid_options:
                time.sleep(1)
                option = op.text.replace('ม.', '')
                # print(f"{option}, {input_data[text]}, {option == input_data[text]}")

                if option == input_data[text]:
                    selected_option = op
                    print(f'เจอ {input_data[text]} selected_option: {op.text}')
                    break
            
            # ถ้าพบตัวเลือกที่ตรงกับค่าจาก .env ให้เลือก
            if selected_option:
                driver.execute_script("arguments[0].scrollIntoView();", selected_option)
                driver.execute_script("arguments[0].click();", selected_option)
                print(f"✅ เลือก {input_data['ระดับชั้น']} ในช่อง {text}")
            else:
                print(f"⚠️ ไม่พบค่าที่ตรงกันใน dropdown สำหรับ '{text}'")
        
        time.sleep(1)

📌 พบ Dropdown ในช่อง: คำนำหน้าชื่อ
เจอ นางสาว selected_option: นางสาว
✅ เลือก 5/1 ในช่อง คำนำหน้าชื่อ
📌 พบ Dropdown ในช่อง: ระดับชั้น
เจอ 5/1 selected_option: ม.5/1
✅ เลือก 5/1 ในช่อง ระดับชั้น


In [6]:
# กดปุ่มถัดไป เพื่อไปหน้าทำข้อสอบ
want_click_name = 'ถัดไป'
try:
    submit_button = driver.find_elements(By.CSS_SELECTOR, "div[role='button']")
    i = 0
    for btn in submit_button:
        if btn.text.strip() == want_click_name:  # เช็คข้อความปุ่มที่ trim ช่องว่าง
            driver.execute_script("arguments[0].click();", btn)
            print(f"\n###########📩 กดปุ่ม '{want_click_name}' สำเร็จ ###########\n")
            i+=1
    if i <= 0:
        print(f"❌ไม่พบปุ่ม '{want_click_name}'")

except Exception as e:
    print(e)


###########📩 กดปุ่ม 'ถัดไป' สำเร็จ ###########

Message: stale element reference: stale element not found
  (Session info: chrome=134.0.6998.35); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6227CDF85+26693]
	(No symbol) [0x00007FF62272EAD0]
	(No symbol) [0x00007FF6225B91CA]
	(No symbol) [0x00007FF6225CF304]
	(No symbol) [0x00007FF6225CDDD3]
	(No symbol) [0x00007FF6225C1869]
	(No symbol) [0x00007FF6225BF632]
	(No symbol) [0x00007FF6225C38FC]
	(No symbol) [0x00007FF6225C39CF]
	(No symbol) [0x00007FF622609386]
	(No symbol) [0x00007FF6226378EA]
	(No symbol) [0x00007FF622602096]
	(No symbol) [0x00007FF622637B00]
	(No symbol) [0x00007FF6226600B8]
	(No symbol) [0x00007FF6226376C3]
	(No symbol) [0x00007FF622600490]
	(No symbol) [0x00007FF622601743]
	GetHandleVerifier [0x00007FF622B2436D+3525677]
	GetHandleVerifier [0x00007FF622B37F3B+3606523]
	G

In [7]:
"""

นำคำตอบจาก JSON มากรอก Input Radio, Dropdown

"""
import re

# ดึงข้อมูลหน้าเว็บ
data = driver.page_source
soup = bs4.BeautifulSoup(data, "html.parser")
list_items = soup.select('div[role=listitem]')

# โหลด JSON
with open('correct_answers.json', 'r', encoding='utf-8') as file:
    correct_answers = json.load(file)

# สร้าง dictionary ของคำตอบ
answer_dict = {item['question']: item['anwser'].strip() for item in correct_answers}

# วนลูปผ่านแต่ละข้อในหน้าเว็บ
for index, item in enumerate(list_items, 1):
    try:
        print(f"กำลังตรวจสอบข้อที่ {index}")
        
        # ดูว่ามีคำถามในข้อนี้หรือไม่
        question_elem = item.select_one('div[role="heading"]')
        if not question_elem:
            print(f"⚠️ ไม่พบหัวข้อคำถามในข้อที่ {index}")
            continue
            
        # ดึงหมายเลขข้อจากข้อความคำถาม
        question_text = question_elem.get_text().strip()
        question_num_match = re.search(r'(\d+)\.', question_text)
        
        if question_num_match:
            question_num = int(question_num_match.group(1))
            print(f"พบคำถามข้อที่: {question_num}")
            
            # ค้นหาคำตอบจาก dictionary
            if question_num in answer_dict:
                correct_answer_text = answer_dict[question_num]
                print(f"คำตอบที่ถูกต้องคือ: {correct_answer_text}")
                
                # หาตัวเลือกทั้งหมด
                radio_options = item.select('div.Od2TWd.hYsg7c')
                
                # ตรวจสอบว่ามีตัวเลือกหรือไม่
                if not radio_options:
                    print("⚠️ ไม่พบตัวเลือกในข้อนี้")
                    continue
                
                # หาตัวเลือกที่ตรงกับคำตอบที่ถูกต้อง
                found_match = False
                for radio in radio_options:
                    label_text = radio.get('aria-label', '').strip()
                    
                    # เช็คว่าตัวเลือกนี้ตรงกับคำตอบหรือไม่
                    if label_text == correct_answer_text or correct_answer_text.startswith(label_text[:2]):
                        # ใช้ selenium คลิกเลือก
                        radio_id = radio.get('id')
                        if radio_id:
                            try:
                                driver.find_element(By.ID, radio_id).click()
                                print(f"✅ เลือกตัวเลือก ID: {radio_id} - {label_text}")
                                found_match = True
                                break
                            except Exception as e:
                                print(f"⚠️ ไม่สามารถคลิกตัวเลือก: {str(e)}")
                
                if not found_match:
                    print(f"⚠️ ไม่พบตัวเลือกที่ตรงกับคำตอบ: {correct_answer_text}")
            else:
                print(f"⚠️ ไม่พบคำตอบสำหรับข้อที่ {question_num} ในไฟล์ JSON")
        else:
            print(f"⚠️ ไม่สามารถระบุหมายเลขข้อจากคำถาม: {question_text}")
    
        time.sleep(1)
        
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาด: {str(e)}")

# try:
#     with open('correct_answers.json', 'r', encoding="utf-8") as file:
#         correct_answers = json.load(file)
        
#         for answer in correct_answers:
#             print(f"Question: {answer['question']}")
#             print(f"Correct Answer: {answer['anwser']}")
#             print('-' * 50)

# except FileNotFoundError:
#     print("File 'correct_answers.json' not found.")

กำลังตรวจสอบข้อที่ 1
⚠️ ไม่สามารถระบุหมายเลขข้อจากคำถาม: คำชี้แจง : ให้นักเรียนเลือกคำตอบที่ถูกต้องที่สุดเพียงคำตอบเดียว
กำลังตรวจสอบข้อที่ 2
พบคำถามข้อที่: 1
คำตอบที่ถูกต้องคือ: ง. การช่วยเหลือผู้ที่หยุดหายใจหรือหัวใจหยุดเต้น ให้มีการหายใจและการไหลเวียนเลือดกลับคืนสู่สภาพปกติ
✅ เลือกตัวเลือก ID: i15 - ง. การช่วยเหลือผู้ที่หยุดหายใจหรือหัวใจหยุดเต้น ให้มีการหายใจและการไหลเวียนเลือดกลับคืนสู่สภาพปกติ
กำลังตรวจสอบข้อที่ 3
พบคำถามข้อที่: 2
คำตอบที่ถูกต้องคือ: ข. CPR
✅ เลือกตัวเลือก ID: i26 - ข. CPR
กำลังตรวจสอบข้อที่ 4
พบคำถามข้อที่: 3
คำตอบที่ถูกต้องคือ: ง. Respiratory Arrest
✅ เลือกตัวเลือก ID: i49 - ง. Respiratory Arrest
กำลังตรวจสอบข้อที่ 5
พบคำถามข้อที่: 4
คำตอบที่ถูกต้องคือ: ก. ผายปอด
✅ เลือกตัวเลือก ID: i57 - ก. ผายปอด
กำลังตรวจสอบข้อที่ 6
พบคำถามข้อที่: 5
คำตอบที่ถูกต้องคือ: ข. 4 นาที
✅ เลือกตัวเลือก ID: i77 - ข. 4 นาที
กำลังตรวจสอบข้อที่ 7
พบคำถามข้อที่: 6
คำตอบที่ถูกต้องคือ: ค. C-A-B
✅ เลือกตัวเลือก ID: i97 - ค. C-A-B
กำลังตรวจสอบข้อที่ 8
พบคำถามข้อที่: 7
คำตอบที่ถูกต้องคือ: ง. 

In [8]:
# กดปุ่มส่ง เพื่อรับ 10คะแนนแบบโปรๆ
want_click_name = 'ส่ง'
try:
    submit_button = driver.find_elements(By.CSS_SELECTOR, "div[role='button']")
    i = 0
    for btn in submit_button:
        if btn.text.strip() == want_click_name:  # เช็คข้อความปุ่มที่ trim ช่องว่าง
            driver.execute_script("arguments[0].click();", btn)
            print(f"\n###########📩 กดปุ่ม '{want_click_name}' สำเร็จ ###########\n")
            i+=1
    if i <= 0:
        print(f"❌ไม่พบปุ่ม '{want_click_name}'")

except Exception as e:
    print(e)


###########📩 กดปุ่ม 'ส่ง' สำเร็จ ###########

Message: stale element reference: stale element not found
  (Session info: chrome=134.0.6998.35); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6227CDF85+26693]
	(No symbol) [0x00007FF62272EAD0]
	(No symbol) [0x00007FF6225B91CA]
	(No symbol) [0x00007FF6225CF304]
	(No symbol) [0x00007FF6225CDDD3]
	(No symbol) [0x00007FF6225C1869]
	(No symbol) [0x00007FF6225BF632]
	(No symbol) [0x00007FF6225C38FC]
	(No symbol) [0x00007FF6225C39CF]
	(No symbol) [0x00007FF622609386]
	(No symbol) [0x00007FF6226378EA]
	(No symbol) [0x00007FF622602096]
	(No symbol) [0x00007FF622637B00]
	(No symbol) [0x00007FF6226600B8]
	(No symbol) [0x00007FF6226376C3]
	(No symbol) [0x00007FF622600490]
	(No symbol) [0x00007FF622601743]
	GetHandleVerifier [0x00007FF622B2436D+3525677]
	GetHandleVerifier [0x00007FF622B37F3B+3606523]
	Get

In [9]:
# กดปุ่มดูคะแนนไปเพื่อแคปหน้าจอ
want_click_name = 'ดูคะแนน'
try:
    submit_button = driver.find_elements(By.CSS_SELECTOR, 'a[aria-label="ดูคะแนน"]')
    i = 0
    for btn in submit_button:
        if btn.text.strip() == want_click_name:  # เช็คข้อความปุ่มที่ trim ช่องว่าง
            driver.execute_script("arguments[0].click();", btn)
            print(f"\n###########📩 กดปุ่ม '{want_click_name}' สำเร็จ ###########\n")
            i+=1
    if i <= 0:
        print(f"❌ไม่พบปุ่ม '{want_click_name}'")

except Exception as e:
    print(e)


###########📩 กดปุ่ม 'ดูคะแนน' สำเร็จ ###########



In [11]:
# สลับแท็บไปแท็บใหม่
time.sleep(1)
# ดึงรายการ window handles ทั้งหมด
tabs = driver.window_handles

# สลับไปยังแท็บใหม่ (แท็บล่าสุด)
driver.switch_to.window(tabs[-1])

print("✅ เปลี่ยนไปที่แท็บใหม่สำเร็จ!")

try:
    # รอให้ Element ปรากฏ
    wait = WebDriverWait(driver, 10)
    score_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "body > div > div:nth-child(2) > div:nth-child(1) > div > div.Dq4amc > div > div.N0gd6 > div.ahS2Le > div.tV8Uvb")))

    # ดึงคะแนน
    score_text = score_element.text
    print(f"คะแนนรวม: {score_text}")

except Exception as e:
    print(f"เกิดข้อผิดพลาด: {e}")

print("\n" + "="*50)
print("🏆ภาระกิจทำข้อสอบอัตโนมัติเสร็จสิ้น💯")
print("="*50 + "\n")

# แคปหน้าจอ
try:
    score = driver.find_elements(By.CSS_SELECTOR, 'div.tV8Uvb')
    driver.save_screenshot("mobile_screenshot.png")
    print("แคปหน้าจอเสร็จแล้ว! บันทึกที่ mobile_screenshot.png")
except Exception as e:
    print(e)

KeyboardInterrupt: 